# Cloud computing
- AWS access: Console, CLI, Boto3
- loading data into buckets
- setting up a reserved instance
- using containers
- distributed computing via pyspark
- run jupyter on ec2 instances

### Amazon web services (AWS)

- No special reason to study Amazon Cloud compared to Google Cloud, Azure, Digital Ocean etc.
- Amazon Cloud is the most popular cloud today. 
- AWS is the web-based gateway to the Amazon Cloud computing resources.
- AWS has a region in Sweden
- AWS is an umbrella for a large number of computing resources, starting from storage and ending with the management of the remote computing infrastructure.
- https://aws.amazon.com/

### Loading data into S3 buckets


- Useful for: backup, sharing research or data (when legal)
- Clouds use the concept of 'buckets' to hold data. 
- The 'objects' stored in a bucket can have any encoding, from text to film.
- The maximum size for one object is 5TB (on AWS).
- Internaly the web console, via the command line interface and Boto3 share the same API.


**using the web console**

Task:
- Use the console to load a test file onto a S3 bucket
- Follow this doc link: https://docs.aws.amazon.com/AmazonS3/latest/user-guide/upload-objects.html
- Use the following shell command to generate some test data, or use your own:
```
$ for i in {1..5}; do echo "l$i">f$i.txt && gzip f$i.txt; done && \
zcat f*.txt.gz| gzip > f.gz
```
- Figure out how much your bucket would cost (tip: it is free up to a threshold)!

**using the CLI (command line interface)**

Now let's repeat those steps using the command line interface. But first, we must install it.

Links: 
- https://docs.aws.amazon.com/cli/latest/userguide/using-s3-commands.html
- https://aws.amazon.com/getting-started/tutorials/backup-to-s3-cli/

```
$ sudo apt install awscli
$ aws configure
AWS Access Key ID [None]: 
AWS Secret Access Key [None]:
(also used eu-central-1 for region, and json as format)
```

The above commang needs SSL certificates. To generate the aws keys:
- https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/set-up-ami-tools.html?icmpid=docs_iam_console#ami-tools-managing-certs

```
$ openssl genrsa 2048 > aws-private.pem
$ openssl req -new -x509 -nodes -sha256 -days 365 -key aws-private.pem -outform PEM -out aws-certificate.pem

# if in dire need for security use:
$ sudo apt-get install xclip
$ xclip -sel clip < ~/.ssh/aws-private.pem
```

Now that you installed the CLI, here are the main bucket related activities:

```
aws s3 mb s3://my-first-backup-bucket
upload:
aws s3 cp “C:\users\my first backup.bak” s3://my-first-backup-bucket/
download:
aws s3 cp s3://my-first-backup-bucket/my-first-backup.bak ./
delete:
aws s3 rm s3://my-first-backup-bucket/my-first-backup.bak
```

Data can also be streamed towards a bucket. This can be useful to avoid unnecesary space waste onto the local cloud or PC, but it can be just as useful when it comes to using bucket data without storing all that data locally. It can be done via piping, or proccess substitution:

```
$ aws s3 mb s3://siofuysni78
$ zcat f*.txt.gz| gzip | aws s3 cp - s3://siofuysni78/f.gz
$ aws s3 rm s3://siofuysni78/f.gz
$ aws s3 rb s3://siofuysni78 --force
```

Why did I use such a weird name? It is because Amazon indexes all buckets by their name, thus a name such as "test123" will never fly. Here is how to stream from S3 to your computing resource (it can be a cloud instance, you local machine or a remore server)

```
$ aws s3 mb s3://siofuysni78
$ zcat f*.txt.gz| gzip | aws s3 cp - s3://siofuysni78/f.gz
$ aws s3 cp s3://siofuysni78/f.gz - | gunzip | grep 1
l1
```


**using Boto3**


Links:
- https://aws.amazon.com/sdk-for-python/
- https://github.com/boto/boto3
- http://boto3.readthedocs.io/en/latest/guide/migration.html#installation-configuration
- https://boto3.readthedocs.io/en/latest/guide/s3-example-creating-buckets.html
- http://boto3.readthedocs.io/en/latest/reference/services/s3.html


```
conda install -c anaconda boto3
pip install boto3
```

In [1]:
import boto3

# initialize the S3 service
s3 = boto3.client('s3')

# create a test bucket (tip: use a different name!)
s3.create_bucket(Bucket='jo8a7fn8sfn8', CreateBucketConfiguration={'LocationConstraint': 'eu-central-1'})

# Call S3 to list current buckets
response = s3.list_buckets()

# Get a list of all bucket names from the response
buckets = [bucket['Name'] for bucket in response['Buckets']]

# Print out the bucket list
print("Bucket List: %s" % buckets)

Bucket List: ['crasstestdummy', 'jo8a7fn8sfn8', 'siofuysni78', 'snlmocombined']


In [ ]:
import boto3

# Create an S3 client
s3 = boto3.client('s3')

filename = '/path/to/test/file'
bucket_name = 'jo8a7fn8sfn8'

# Uploads the given file using a managed uploader, which will split up large
# files automatically and upload parts in parallel.
s3.upload_file(filename, bucket_name, filename)

# or
# s3.Object('mybucket', 'hello.txt').put(Body=open('/tmp/hello.txt', 'rb'))

In [ ]:
# https://boto3.readthedocs.io/en/latest/guide/migrations3.html#deleting-a-bucket
import boto3
import botocore

s3 = boto3.resource('s3')
bucket = s3.Bucket('jo8a7fn8sfn8')

for key in bucket.objects.all():
    key.delete()
bucket.delete()





Now I want to test using the buchet without local file storage.



## Setting up a reserved instance


What can be done on a cloud instance?
- It works as a remote PC, on which you have admin priviledge. You can put anything there, run any program. (make sure it is legal)

Amazon names their most popular instances Elastic Compute Cloud (EC2).
- https://aws.amazon.com/ec2/
- https://docs.aws.amazon.com/efs/latest/ug/gs-step-one-create-ec2-resources.html
- https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/get-set-up-for-amazon-ec2.html

Probably the most basic level of access to the Amazon computing infrastructure is setting up a free tier reserved instance. 

### Web Console

Task:
- Setup an AWS instance using the Free Tier (don't forget to close it!).
- [https://aws.amazon.com/console/](https://aws.amazon.com/console/)


### Amazon CLI


```
aws ec2 run-instances --image-id ami-xxxxxxxx --count 1 --instance-type t1.micro --key-name MyKeyPair --security-groups my-sg
```

    
### Boto3


- http://boto3.readthedocs.io/en/latest/reference/services/ec2.html#instance
- http://boto3.readthedocs.io/en/latest/guide/migrationec2.html#launching-new-instances

Task:
- A larger task is to create an instance with Boto3, install an SSH client such as Paramaiko and run commands on the remote client.

Helpful code:

In [ ]:
import boto3
import botocore
import paramiko


ec2 = boto3.resource('ec2')
instance = ec2.Instance('id')
ec2.create_instances(ImageId='<ami-image-id>', MinCount=1, MaxCount=5)

key = paramiko.RSAKey.from_private_key_file(path/to/mykey.pem)
client = paramiko.SSHClient()
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

# Connect/ssh to an instance
try:
    # Here 'ubuntu' is user name and 'instance_ip' is public IP of EC2
    client.connect(hostname=instance_ip, username="ubuntu", pkey=key)

    # Execute a command(cmd) after connecting/ssh to an instance
    stdin, stdout, stderr = client.exec_command(cmd)
    print stdout.read()

    # close the client connection once the job is done
    client.close()
    break

except Exception, e:
    print e

### When do we need containers? Using Docker.

- Separating the environment from the operating system
- Virtualization, VirtualBox, VMWare
- Containers, Docker, Singularity, Vagrant
- DockerHub
- Backup the code, the data and the environment

**Example of a simple Dockerfile:**

In [ ]:
# Use an official Python runtime as a base image
FROM python:2.7-slim

# Set the working directory to /app
WORKDIR /app

# Copy the current directory contents into the container at /app
ADD . /app

# Install any needed packages specified in requirements.txt
RUN pip install -r requirements.txt

# Make port 80 available to the world outside this container
EXPOSE 80

# Define environment variable
ENV NAME World

# Run app.py when the container launches
CMD ["python", "app.py"]

**Build the image:**

**Run the image:**

**Publish on the docker hub and run on any computer on Earth:**

```
# signup at cloud.docker.com
docker tag friendlyhello username/repository:tag
docker push username/repository:tag
```

Now go to any computer on Earth having Docker installed and access to Internet and run:

```
docker run -p 4000:80 username/repository:tag
```


**Spin up containers via Docker Machine**

My preferred way is to use docker machine in order to manage cloud instances that are already set-up with Docker. Then you can pull your intended container from the Docker Hub and run it on the instance. An alternative is usign AWS services to create your instance, which has its own benefits (basically most benefits except for time). Another alternative is usign Docker Cloud or Kubernetes, which is the way to go for multiple instances.


```
# install docker machine
$ base=https://github.com/docker/machine/releases/download/v0.14.0 && \
curl -L $base/docker-machine-$(uname -s)-$(uname -m) >/tmp/docker-machine && \
sudo install /tmp/docker-machine /usr/local/bin/docker-machine

# setup a cloud instance
$ export SECRET_KEY="..."
$ docker-machine create --driver amazonec2 --amazonec2-region eu-central-1 \
--amazonec2-access-key <your_key> \
--amazonec2-secret-key $SECRET_KEY \
aws-test

# ssh and delete
docker-machine ssh aws-test
docker-machine rm aws-test

# for other options: --amazonec2-instance-type "t2.2xlarge"
docker-machine create --driver amazonec2
```


**Further read**

- On Docker Machine:
    - https://docs.docker.com/machine/examples/aws/
    - https://docker-curriculum.com/
    - https://sreeninet.wordpress.com/2016/09/03/docker-machine-for-aws/
    - http://blog.wimwauters.com/docker-getting-started-with-docker-machine-aws/
    - http://docker-k8s-lab.readthedocs.io/en/latest/docker/docker-machine-aws.html
- Using AWS alternatives:
    - https://aws.amazon.com/getting-started/tutorials/deploy-docker-containers/
    - https://docs.aws.amazon.com/AmazonECS/latest/developerguide/docker-basics.html#docker-basics-create-image
    - https://docs.aws.amazon.com/AmazonECR/latest/userguide/docker-push-ecr-image.html
- Or the more simplified use of Docker cloud, by linking to AWS. There are other, such as Kubernetes, but it is too complex (we only need one instance):
    - https://docs.docker.com/docker-cloud/infrastructure/link-aws/
    - https://docs.docker.com/docker-cloud/getting-started/your_first_node/



### Pull the docker hub container on the EC2 instance, open shell and run test

```
# run inside the EC2 instance
export DOCKER_ID_USER="<your_ID>"
docker login
docker pull $DOCKER_ID_USER/awstest
docker run -ti $DOCKER_ID_USER/awstest /bin/bash

# now run your commands inside the container
```

This works only if you must run short tasks, because once you log out from the container, the container will end. What you need is to be able to run long jobs. So we must create a detached container, then attach to it during and after the execution of the program with a shell in order to check the logs and save data.

```
# run inside the EC2 instance
export DOCKER_ID_USER=""
sudo docker login
sudo docker pull $DOCKER_ID_USER/awscrass
sudo docker run -w /home/ -tid $DOCKER_ID_USER/awscrass /bin/bash

# exit container, start it
sudo docker ps
# run a command in detached mode
#sudo docker exec -d containerid bash -c "your command line"

#alternative is to log into the container and run the command there
sudo docker exec -it containerid bash

# start, attach
docker start containerid
docker attach containerid

```



**Pricing**

Further read:
- autoscaling? https://aws.amazon.com/autoscaling/
- https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/EC2_GetStarted.html
- https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/Stop_Start.html
- https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-instance-resize.html

## Task


Your task will be to create a Docker container, push it to Docker Hub, start an EC2 instance and remotely run your container, log out from the instance, then logging back and checking that the output is preserved.


## Instance Types

On AWS you can opt for different types of instances, and you can also upgrade or downgrade your instance to meet your need for resources. One can opt for example for instances that have a lot of RAM assigned when using a RAM intensive computation such as sequence alignment, or GPU instances when needing deep learning or other forms of GPU accelerated computing. More here: https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/instance-types.html

From the purchasing point of view there are however several major classes of instances, most notably:
- On demand instances. Such instances are available when you request them and will be held up until you close them. - Spot instances. A Spot Instance is an unused EC2 instance that is available for less than the On-Demand price. Your Spot Instance runs whenever capacity is available and the maximum price per hour for your request exceeds the Spot price. 
- read about the other types here: https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/instance-purchasing-options.html




## ECS clusters and Docker Cloud


You can run containerized clusters of EC2 instances using another AWS web service called ECS clusters. More information here: https://docs.aws.amazon.com/AmazonECS/latest/developerguide/ECS_clusters.html

Another popular option is to use Docker Cloud, allowing you to define and store images and build mechanisms of continuous integration then run tests or production clusters on AWS.

Task:
- Learn how to deply an Elastic HPC cluster: https://aws.amazon.com/getting-started/projects/deploy-elastic-hpc-cluster/



# Spark


Further read:
- https://medium.com/big-data-on-amazon-elastic-mapreduce/run-a-spark-job-within-amazon-emr-in-15-minutes-68b02af1ae16
- https://www.themarketingtechnologist.co/upload-your-local-spark-script-to-an-aws-emr-cluster-using-a-simply-python-script/

**Run Spark on EC2 instances**


Go to EMR and pick the cheapest option for your region, optimized for computing, mine is c5.xlarge.
- https://aws.amazon.com/emr/pricing/?nc1=h_ls
- https://aws.amazon.com/s3/pricing/

The region settings for Stockholm can be found at:
https://docs.aws.amazon.com/general/latest/gr/rande.html#apigateway_region
I am using the Stockholm region (eu-north-1).

Next you need access keys for IAM roles. Open https://console.aws.amazon.com/iam/ and folow the instructions:
- https://docs.aws.amazon.com/cli/latest/userguide/cli-chap-configure.html

In [ ]:
# used from https://github.com/apache/spark/blob/master/examples/src/main/python/pi.py

from __future__ import print_function

import sys
from random import random
from operator import add

from pyspark.sql import SparkSession


if __name__ == "__main__":
    """
        Usage: pi [partitions]
    """
    spark = SparkSession\
        .builder\
        .appName("PythonPi")\
        .getOrCreate()

    partitions = int(sys.argv[1]) if len(sys.argv) > 1 else 2
    n = 100000 * partitions

    def f(_):
        x = random() * 2 - 1
        y = random() * 2 - 1
        return 1 if x ** 2 + y ** 2 <= 1 else 0

    count = spark.sparkContext.parallelize(range(1, n + 1), partitions).map(f).reduce(add)
    print("Pi is roughly %f" % (4.0 * count / n))

    spark.stop()

You can run cluster jobs interactively by SSH to the master node. But there are many other ways in which you can run your jobs on the spark cluster. Complete freedom!

You can also add steps via the awscli and monitor them online:

```
$ aws emr add-steps \
--cluster-id j-3H0XTRI8687P2 \
--steps Type=CUSTOM_JAR,Name="Spark Program",Jar="command-runner.jar",ActionOnFailure=CONTINUE,Args=["spark-submit",/home/hadoop/pitest.py]
```

**SUPER IMPORTANT STEP..**

```
aws emr terminate-clusters --cluster-ids j-3H0XTRI8687P2
```

OBS: Verify on https://eu-north-1.console.aws.amazon.com/elasticmapreduce/ that your cluster was terminated properly!

Task:
- Log the result in an s3 bucket.
- Try to load a data set and perform a basic ML task!
- Configure JupyterHub via EMR, load giant pyspark steps from the comfort of your phone's web browser. Profit! ;)


Task:
- Run Jupyter inside an EC2 instance. Examples:
    - https://dataschool.com/data-modeling-101/running-jupyter-notebook-on-an-ec2-server/
    - https://medium.com/@josemarcialportilla/getting-spark-python-and-jupyter-notebook-running-on-amazon-ec2-dec599e1c297